<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/3_rag_texto_custom_semantic_chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruções

* Estratégia 1
    * Usar segmentação de sentenças
    * BM25 como buscador

# Instalando pacotes

In [ ]:
!pip install -q llama-index llama-index-llms-groq llama-index-retrievers-bm25 sentence-transformers llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Importando pacotes

In [ ]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [ ]:
import json
import time
import random
from tqdm import tqdm
from rich.pretty import pprint

from llama_index.core import Document
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.callbacks import (CallbackManager, LlamaDebugHandler, CBEventType)
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.response.notebook_utils import display_source_node, display_response
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent

from llama_index.llms.groq import Groq

from IPython.display import Markdown, display
import pickle
import pandas as pd

In [ ]:
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

Cloning into 'ia024-projeto-rag'...
remote: Enumerating objects: 6045, done.
remote: Counting objects: 100% (1050/1050), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 6045 (delta 658), reused 1050 (delta 658), pack-reused 4995
Receiving objects: 100% (6045/6045), 1.12 GiB | 24.29 MiB/s, done.
Resolving deltas: 100% (2047/2047), done.
Updating files: 100% (5787/5787), done.


In [ ]:
DATA_DIR = '/content/ia024-projeto-rag'

# Funções

In [ ]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [ ]:
%cd /content/ia024-projeto-rag

!git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

!git pull origin main

/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.


# Criando BM25

## Criando documentos do llama-index

In [ ]:
from llama_index.core.schema import MetadataMode

docs = []

with open(f"{DATA_DIR}/2-dataset/estatuto.json", 'r') as f:
    estatuto = json.load(f)

metadata={
        "doc_name": estatuto['titulo'],
        "category": "Reitoria"
    }

doc = Document(id_=0
               , text=estatuto['texto_compacto']
               , metadata=metadata
               )

#print(
#    "The LLM sees this: \n",
#    doc.get_content(metadata_mode=MetadataMode.LLM),
#)
#print(
#    "The Embedding model sees this: \n",
#    doc.get_content(metadata_mode=MetadataMode.EMBED),
#)

docs.append(doc)

In [ ]:
pprint(docs[0])

Document(
│   id_='0',
│   embedding=None,
│   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={},
│   text='ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO I - DA NATUREZA E DAS FINALIDADES. Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I. Reitoria, no endereço citado no §1º. II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo. III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão. IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho. V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos. VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista. VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba. VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista. IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto. X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque. XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos. XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão. XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos. XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano. XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais, Campinas; XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva. XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré. XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara. XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga. XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui. XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga. XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro. XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio. XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba. XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia. XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva. XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari. XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão. XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos. XXXI. Núcleo Avançado de Assis, situado na Avenida Getúlio Vargas nº 1200, Vila Nova Santana, Assis. § 3º- Para efeito da incidência das disposições qu

## Fazendo janelamento

In [ ]:
# Esta classe separa o texto em sentenças e, de acordo com o parâmetro "window_size",
# cria janelas com a sentença atual mais 'window_size' sentenças antes e depois
# as janelas são salvas no metadata "window" e as sentenças em "original_sentence"
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3, # com valor 1, as "janelas" terão tamanho 3: uma sentença antes, a sentença atual, uma sentença depois
    window_metadata_key="window",
    original_text_metadata_key="original_sentence",
)

nodes = node_parser.get_nodes_from_documents(docs)

In [ ]:
len(nodes)

264

In [ ]:
pprint(nodes[0])

TextNode(
│   id_='d53e388f-db6f-4207-9794-f15b5d01d0c7',
│   embedding=None,
│   metadata={
│   │   'window': 'ESTATUTO.  TÍTULO I - DA INSTITUIÇÃO.  CAPÍTULO I - DA NATUREZA E DAS FINALIDADES.  Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. ',
│   │   'original_sentence': 'ESTATUTO. ',
│   │   'doc_name': 'Estatuto',
│   │   'category': 'Reitoria'
│   },
│   excluded_embed_metadata_keys=['window', 'original_sentence'],
│   excluded_llm_metadata_keys=['window', 'original_sentence'],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   node_id='0',
│   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   hash='c958262a924fc6e53db65cce9967b79c93c611568eb1a613ff6791791d3dfe21'
│   │   ),
│   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
│   │   │   node_id='f5f9d40b-c8bb-4370-9998-58045fc05749',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={
│   │   │   │   'window': 'ESTATUTO.  TÍTULO I - DA INSTITUIÇÃO.  CAPÍTULO I - DA NATUREZA E DAS FINALIDADES.  Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº.  11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. ',
│   │   │   │   'original_sentence': 'TÍTULO I - DA INSTITUIÇÃO. '
│   │   │   },
│   │   │   hash='8f22f8fe4aba5ca0baa7c6c5cbd7ec7b57a0cf66c22597581eaff1c5e2d7c1d0'
│   │   )
│   },
│   text='ESTATUTO. ',
│   start_char_idx=0,
│   end_char_idx=10,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

## Fazendo Semantic Chunking

https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_chunking/

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SemanticSplitterNodeParser

embed_model = HuggingFaceEmbedding(model_name="alfaneo/bertimbau-base-portuguese-sts")

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

nodes_semantic = splitter.get_nodes_from_documents(docs)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.39k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/438k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
len(nodes_semantic)

15

In [ ]:
pprint(nodes_semantic[1])

TextNode(
│   id_='299a0109-dc25-403f-8c4d-3c5f8349bbc2',
│   embedding=None,
│   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   node_id='0',
│   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   hash='c958262a924fc6e53db65cce9967b79c93c611568eb1a613ff6791791d3dfe21'
│   │   ),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
│   │   │   node_id='da0da63a-6cdf-4414-a585-aec150df7977',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   hash='d0d5e2b11df42ecd9d9541e036be4b4e9a90e70d80fa688d60ee087f4c9a52d6'
│   │   ),
│   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
│   │   │   node_id='13e08cf9-f056-493f-be14-908eb40223ea',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={},
│   │   │   hash='2f2535b911ba29aefc743a6708f566046c84d0e7d853fd0e6cce422dad318bab'
│   │   )
│   },
│   text='TÍTULO I - DA INSTITUIÇÃO. ',
│   start_char_idx=10,
│   end_char_idx=37,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

In [ ]:
nodes_semantic[2].get_content()

'CAPÍTULO I - DA NATUREZA E DAS FINALIDADES. Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I. Reitoria, no endereço citado no §1º. II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo

# Custom semantic chuncking

In [ ]:
# Custom Splitter Function
import re
from typing import List

def custom_article_splitter(text: str) -> List[str]:
    article_pattern = re.compile(r'\bArt\.?\s\d+\b', re.IGNORECASE)
    splits = article_pattern.split(text)
    articles = []

    for i, split in enumerate(splits):
        if i == 0:
            articles.append(split.strip())
        else:
            article_number = article_pattern.findall(text)[i - 1]
            articles.append(f"Art {article_number} {split.strip()}")

    return articles

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="alfaneo/bertimbau-base-portuguese-sts")

# Define the semantic splitter with the custom article splitter
class CustomSemanticSplitter(SemanticSplitterNodeParser):
    def __init__(self, sentence_splitter, embed_model):
        super().__init__(sentence_splitter=sentence_splitter, embed_model=embed_model)

semantic_splitter = CustomSemanticSplitter(sentence_splitter=custom_article_splitter, embed_model=embed_model)

In [ ]:
base_nodes = semantic_splitter.get_nodes_from_documents(docs)

# Assign node IDs
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

# Create the Vector Store Index and Base Retriever
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq

embed_model = HuggingFaceEmbedding(model_name="alfaneo/bertimbau-base-portuguese-sts")
llm = Groq(model="llama3-70b-8192")

base_index = VectorStoreIndex(base_nodes, embed_model=embed_model)
base_retriever = base_index.as_retriever(similarity_top_k=2)

# Set Up Recursive Retriever
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [SentenceSplitter(chunk_size=c, chunk_overlap=20) for c in sub_chunk_sizes]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes]
        all_nodes.extend(sub_inodes)
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

all_nodes_dict = {n.node_id: n for n in all_nodes}

In [ ]:
vector_index_chunk = VectorStoreIndex(all_nodes, embed_model=embed_model)
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=15)

from llama_index.core.retrievers import RecursiveRetriever

retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

# Integrate with RetrieverQueryEngine and Metadata Postprocessing
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever_chunk,
    llm=Groq(model="llama3-70b-8192"),
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ]
)

In [ ]:
# Query the engine
response = query_engine.query(
    #  "Qual o nome do IFSP?"                                                                  # Conseguiu Responder
    #  "Liste quais são os campi do IFSP?"                                                           # Respondeu parcialmente, pois não listou todos os campi
    #  "O IFSP tem só ensino médio? Informe o documento, o Capítulo e o Artigo de referência." # Só conseguiu falar que é o Estatuto e o Art 5, mas errou o Capítulo
    #  "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência." # Respondeu, dizendo o Documento, o Capítulo e o Artigo (Art. 1)
     "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"              # Menciona que o Art. 32 fala sobre educação superior de graduação e pós-graduação
    #  "O que diz o Art 6 do estatuto do IFSP?"
    )

# Display the response|
print(response)

In [ ]:
save_on_git("Implementing a custom semantic chuncking")

/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


## Criando BM25

In [ ]:
# Criando o BM25
# com 'tokenizer=None', ele vai fazer a remoção de stop words e o stemming
retriever = BM25Retriever(nodes=nodes, tokenizer=None, similarity_top_k=10)

In [ ]:
# Testando a busca com BM25
retriever_nodes = retriever.retrieve("Qual o nome do IFSP?")
for node in retriever_nodes:
    display_source_node(node)

**Node ID:** 5cc235b3-f865-422d-958d-0d3a1c055878<br>**Similarity:** 0.0<br>**Text:** §8º- Os procedimentos de trabalho do Conselho Superior serão disciplinados no seu regulamento o q...<br>

**Node ID:** 5374a138-a569-4ac4-ac64-e3a8ba044ea0<br>**Similarity:** 0.0<br>**Text:** Subseção I: Da Auditoria Interna Art14 - A Unidade de Auditoria Interna é órgão de assessoramento...<br>

**Node ID:** 3f62b5a4-1c62-4b69-a4f6-d403da8975a1<br>**Similarity:** 0.0<br>**Text:** Atos Administrativos do IFSP.<br>

**Node ID:** d700f906-a83d-415a-af53-d934517735b2<br>**Similarity:** 1.0041925000614276<br>**Text:** As competências específicas estão definidas no Regimento-Geral do IFSP.<br>

**Node ID:** 752d9393-d694-44c5-a826-e44b6e73b4a3<br>**Similarity:** 0.0<br>**Text:** apreciar os assuntos de interesse da administração do IFSP a ele submetido.<br>

**Node ID:** 48f69994-2183-40eb-b2eb-e21c5ee37710<br>**Similarity:** 1.1821298093422055<br>**Text:** Art 8º - A organização geral do IFSP compreende: I. ÓRGÃOS SUPERIORES a) Conselho Superior; 1.<br>

**Node ID:** 4b02ecbc-7c51-4cd9-a688-6a63a7fd7b57<br>**Similarity:** 0.787010506601938<br>**Text:** Art 3º - Os atos administrativos do IFSP obedecerão à forma de: I. Resolução; II.<br>

**Node ID:** 3c81c03b-a4b1-49d5-8e40-9cb6007f0a8e<br>**Similarity:** 0.0<br>**Text:** Art 44 - O IFSP expedirá e registrará seus diplomas em conformidade com o §3º do art.<br>

**Node ID:** ffad13a8-3d8e-4b6d-a681-f7fb82ba81e8<br>**Similarity:** 0.0<br>**Text:** propor a criação e alteração de funções e órgãos administrativos da estrutura organizacional do I...<br>

**Node ID:** 5dc6722e-79ac-4fe9-b66b-1102af22bfa1<br>**Similarity:** 0.0<br>**Text:** Art 46 - O IFSP poderá conferir títulos de Mérito Acadêmico, conforme disciplinado no Regimento-G...<br>

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

Settings.embed_model = embed_model

vector_index = VectorStoreIndex(nodes_semantic)
query_engine = vector_index.as_query_engine(llm=Groq(model="llama3-70b-8192"))

In [ ]:
response = query_engine.query(
     #"Qual o nome do IFSP?"                                                                  # Conseguiu Responder
     #"Quais são os campi do IFSP?"                                                           # Respondeu parcialmente, pois não listou todos os campi
     "O IFSP tem só ensino médio? Informe o documento, o Capítulo e o Artigo de referência." # Só conseguiu falar que é o Estatuto e o Art 5, mas errou o Capítulo
     #"O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência." # Respondeu, dizendo o Documento, o Capítulo e o Artigo (Art. 1)
     #"Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"              # Menciona que o Art. 32 fala sobre educação superior de graduação e pós-graduação
     #"O que diz o Art 6 do estatuto do IFSP?"                                                 # Não conseguiu responder
)
display_response(response)

**`Final Response:`** According to the provided context, the answer is no, the IFSP does not only have high school education. 

The reference is: Document: Estatuto, Chapter: I, Article: 1, §2º.

# Criando o 'motor de busca'

In [ ]:
# Este motor de busca será utilizado como tool pelo agente

# Após testes e análise do código, percebi que:
# - O BM25 faz a busca utilizando a sentença e não a janela
# - O resultado da busca é substituído pelas respectivas "janelas"
# - O LLM indicado é utilizado para "sintetizar" o conteúdo das janelas
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    llm=Groq(model="llama3-70b-8192"),
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ]
)

In [ ]:
# Testando a busca
response = query_engine.query(
    "Qual o nome do IFSP?"
    #"Quais são os campi do IFSP?"
)
display_response(response)

**`Final Response:`** Não há menção explícita ao nome do IFSP no contexto fornecido.

In [ ]:
for node in response.source_nodes:
    display_source_node(node, source_length=500)

**Node ID:** 4a3fb8d5-d713-47f0-87b3-e2c4258b4dc6<br>**Similarity:** 0.0<br>**Text:** §5º- Ocorrendo o afastamento definitivo de qualquer membro do Conselho Superior, assumirá o respectivo suplente para a complementação do mandato originalmente estabelecido, realizando-se nova eleição para a escolha de suplentes.  §6º- Os membros relacionados nos incisos II a V que se enquadrarem em mais de uma categoria somente poderão se candidatar na categoria mais recente no IFSP.  §7°- Os membros relacionados nos incisos II e IV não poderão ocupar, concomitantemente, cargos de confiança d...<br>

**Node ID:** 9f94ee2f-db1e-4ca6-aa23-1c42a65c2537<br>**Similarity:** 0.0<br>**Text:** deliberar sobre taxas, emolumentos e contribuições por prestação de serviços em geral a serem cobrados pelo IFSP; IX.  autorizar a criação, a alteração curricular e a extinção de cursos no âmbito do IFSP, bem como o registro de diplomas; X. aprovar a estrutura administrativa e o Regimento-Geral do IFSP, observados os parâmetros definidos pelo Governo Federal e pela legislação específica; XI.  deliberar sobre questões submetidas à sua apreciação.  Subseção I: Da Auditoria Interna Art14 - A Uni...<br>

**Node ID:** cfe556e3-4655-4ce6-aa28-1810d90b5e78<br>**Similarity:** 0.0<br>**Text:** III.  Regimento dos campi.  IV.  Atos Administrativos do IFSP.  Art 3º - Os atos administrativos do IFSP obedecerão à forma de: I. Resolução; II.  Parecer; III.  Portaria; IV.<br>

**Node ID:** 8f423bbf-6d4c-49f7-8a96-80b42544a813<br>**Similarity:** 1.0041925000614276<br>**Text:** Art 19 – Os Órgãos Colegiados correspondem aos Conselhos de Ensino, de Pesquisa e Inovação, e de Extensão.  Os respectivos Conselhos são órgãos consultivos, subordinados às diretrizes do Conselho Superior, com funções de supervisão nas matérias de ensino, pesquisa e extensão.  São presididos por seus Pró-Reitores e têm sua composição e funcionamento definidos em Regulamento Próprio.  As competências específicas estão definidas no Regimento-Geral do IFSP.  CAPÍTULO III - DA REITORIA.  Art 20 -...<br>

**Node ID:** 80a4841b-b4c5-4f00-b62f-cdbc9168ff9a<br>**Similarity:** 0.0<br>**Text:** apreciar e recomendar as normas para celebração de acordos, convênios e contratos, bem como para elaboração de cartas de intenção ou de documentos equivalentes; III.  propor a criação e alteração de funções e órgãos administrativos da estrutura organizacional do IFSP; IV.  Apreciar e recomendar o calendário de referência anual; V. apreciar e recomendar normas de aperfeiçoamento da gestão; VI.  apreciar os assuntos de interesse da administração do IFSP a ele submetido.  CAPÍTULO II - DOS ÓRGÃO...<br>

**Node ID:** 3a00c4b2-6e37-4efc-968e-edb36d5d0e99<br>**Similarity:** 1.1821298093422055<br>**Text:** 8º da Lei nº.  11.892/2008.  CAPÍTULO III - DA ORGANIZAÇÃO ADMINISTRATIVA.  Art 8º - A organização geral do IFSP compreende: I. ÓRGÃOS SUPERIORES a) Conselho Superior; 1.  Órgão de Controle: Unidade Auditoria Interna; 2.  Comitê Técnico-Profissional.  b) Colégio de Dirigentes.<br>

**Node ID:** a661a619-db69-410a-8678-f0df2d253847<br>**Similarity:** 0.787010506601938<br>**Text:** Regimento dos campi.  IV.  Atos Administrativos do IFSP.  Art 3º - Os atos administrativos do IFSP obedecerão à forma de: I. Resolução; II.  Parecer; III.  Portaria; IV.  Instrução Normativa; V. Comunicado.<br>

**Node ID:** 11b8000f-ee89-4636-9569-d77a1f0ff61c<br>**Similarity:** 0.0<br>**Text:** Art 42 - O regime disciplinar do corpo discente é estabelecido em regulamento próprio aprovado pelo Conselho Superior.  Art 43 - O regime disciplinar do corpo docente e técnico-administrativo do IFSP observa as disposições legais, normas e regulamentos sobre a ordem disciplinar e sanções aplicáveis, bem como os recursos cabíveis, previstos pela legislação federal.  TÍTULO V - DOS DIPLOMAS, CERTIFICADOS E TÍTULOS.  Art 44 - O IFSP expedirá e registrará seus diplomas em conformidade com o §3º d...<br>

**Node ID:** 096eab73-77fe-4c69-bb32-e2a9a513a012<br>**Similarity:** 0.0<br>**Text:** Parágrafo Único - No impedimento do Reitor, a presidência do Colégio de Dirigentes será exercida pelo seu substituto legal designado na forma da legislação pertinente.  Art 18 - Ao Colégio de Dirigentes compete: I. apreciar e recomendar a distribuição interna de recursos; II.  apreciar e recomendar as normas para celebração de acordos, convênios e contratos, bem como para elaboração de cartas de intenção ou de documentos equivalentes; III.  propor a criação e alteração de funções e órgãos adm...<br>

**Node ID:** 8ff211c8-4480-46fd-891d-a62a61325912<br>**Similarity:** 0.0<br>**Text:** 2º.  da Lei n.º 11.892/2008 e emitirá certificados a alunos concluintes de cursos e programas.  Art 45 - No âmbito de sua atuação, o IFSP funciona como instituição acreditadora e certificadora de competências profissionais, nos termos da legislação vigente.  Art 46 - O IFSP poderá conferir títulos de Mérito Acadêmico, conforme disciplinado no Regimento-Geral.  TÍTULO VI - DO PATRIMÔNIO.  Art 47 - O patrimônio do IFSP é constituído por: I. bens e direitos que compõem o patrimônio da Reitoria e...<br>

In [ ]:
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} " f"\n\n**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))


display_prompt_dict(query_engine.get_prompts())

# Criando Agente

In [ ]:
# Definindo as tools disponíveis
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="search",
            description=(
                "Provides context to answer the questions. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
# Criando o agente
llm   = Groq(model="llama3-70b-8192")
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True # mostra as mensagens na saída
)

In [ ]:
# Testando o agente

agent.reset() # apaga o histórico para executar a próxima consulta

response = agent.chat(
        #"Qual o nome do IFSP?"                                                                  # Conseguiu Responder
        #"Quais são os campi do IFSP?"                                                           # Respondeu parcialmente, pois não listou todos os campi
        #"O IFSP tem só ensino médio? Informe o documento, o Capítulo e o Artigo de referência." # Só conseguiu falar que é o Estatuto e o Art 5, mas errou o Capítulo
        #"O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência." # Respondeu, mas não falou a referência
        #"Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"              # Estourou o número de interações e não conseguiu responder
        "O que diz o Art 6º do estatuto do IFSP?"                                                # Conseguiu responder com 'Art 6º', mas não com "artigo 6" ou "Art 6"
    )

display_response(response, show_source=False)

Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: search
Action Input: {'input': 'Art 6º do estatuto do IFSP'}
Observation: O IFSP tem os seguintes objetivos: I. ministrar educação profissional técnica de nível médio, prioritariamente na forma de cursos integrados, para os concluintes do ensino fundamental e para o público da Educação de Jovens e Adultos; II. ministrar cursos de formação inicial e continuada de trabalhadores, tendo como objetivo a capacitação, o aperfeiçoamento, a especialização e a atualização de profissionais, em todos os níveis de escolaridade, nas áreas da educação profissional e tecnológica; III. realizar pesquisas, estimulando o desenvolvimento de soluções técnicas e tecnológicas, estendendo seus benefícios à comunidade; IV. desenvolver atividades de extensão de acordo com os princípios e finalidades da educação profissional e tecnológica, em articulação com o mundo do trabalho e os segmentos so

**`Final Response:`** O Art. 6º do estatuto do IFSP apresenta os objetivos institucionais, que incluem ministrar educação profissional, realizar pesquisas, desenvolver atividades de extensão, estimular processos educativos, desenvolver programas de extensão e divulgação cultural científica e tecnológica, realizar e estimular a pesquisa, produção cultural, empreendedorismo, cooperativismo e desenvolvimento científico e tecnológico, e promover a produção, desenvolvimento e transferência de tecnologias sociais.

In [ ]:
display_prompt_dict(agent.get_prompts())

**Prompt Key**: agent_worker:system_prompt 

**Text:** 

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 

# Executando e avaliando o agente

In [ ]:
def get_agent_reasoning(agent):
    current_reasoning = agent.list_tasks()[0].extra_state['current_reasoning']
    reasoning = ''
    for item in current_reasoning:
        reasoning += f"* {item.get_content()}\n"

    return reasoning

## Executando o agente

In [ ]:
# Caminho para o arquivo de salvamento
save_file = "qa_data.pickle"

# Verifica se existe um arquivo de salvamento e carrega os dados, se houver
if os.path.exists(save_file):
    with open(save_file, 'rb') as f:
        qa = pickle.load(f)
else:
    qa = {'question': [], 'answer': [], 'agent_answer': [], 'agent_reasoning': []}

# Variável para controlar o índice do último item processado com sucesso
last_processed_index = 0
with tqdm(total=len(test_set)) as pbar:
    while last_processed_index < len(test_set):

        q = test_set[last_processed_index]

        # Se a pergunta já estiver na lista, pule para a próxima
        if q['question'] in qa['question']:
            last_processed_index += 1
            pbar.update(1)
            continue

        try:
            question        = q['question']
            question_answer = format_answer(q)

            # Apaga o histórico do agent para executar a próxima consulta
            agent.reset()

            response = agent.chat(question)

            qa['question'].append(question)
            qa['answer'].append(question_answer)
            qa['agent_answer'].append(response.response)
            qa['agent_reasoning'].append(get_agent_reasoning(agent))

            # Após um número específico de iterações,
            # salva os dados em disco e faz uma pausa para evitar estourar o rate limit
            if len(qa['question']) % 5 == 0:
                with open(save_file, 'wb') as f:
                    pickle.dump(qa, f)

                sleep_time = random.uniform(1, 10)
                time.sleep(sleep_time)

            last_processed_index += 1
            pbar.update(1)

        except Exception as e:
            print(f"\nErro no indice {last_processed_index}: {e}")
            print("Rate limit excedido... esperando 60s")
            time.sleep(60)

# No final do loop, salva os dados restantes em disco
with open(save_file, 'wb') as f:
    pickle.dump(qa, f)

## Avaliando

In [ ]:
with open(save_file, 'rb') as f:
    qa = pickle.load(f)

df = pd.DataFrame(qa)
df.to_csv('resultado.csv', index=False)
df

,question,answer,agent_answer,agent_reasoning
0,What is Zeus know for in Greek mythology?,sky and thunder god,Zeus is known for being the god of the sky and...,* Thought: The current language of the user is...
1,How long had the First World War been over whe...,5 years,"The First World War ended on November 11, 1918.",* Thought: The current language of the user is...
2,How old was Messe when the First World War sta...,30 years,Jesus Christ would be approximately 1918 years...,* Thought: The current language of the user is...
3,How long had Angela Scoular been acting profes...,2 years,Angela Scoular had been acting professionally ...,* Thought: The current language of the user is...
4,What is the capacity of the stadium where Brun...,"26,688",The capacity of the stadium where Brunt return...,* Thought: The current language of the user is...
5,Which stadium where Brunt played can hold more...,White Hart Lane,White Hart Lane can hold more people.,* Thought: The current language of the user is...
6,In which country was Wilhelm Müller born?,Germany,"Wilhelm Müller was born in Gleißenberg, Germany.",* Thought: The current language of the user is...
7,Which battle Wilhelm Müller fought in while in...,Kulm,"Wilhelm Müller was a poet, not a military pers...",* Thought: The current language of the user is...
8,How much time had passed between Wilhelm's ret...,9 years,"Wilhelm returned to studies at Berlin in 1859,...",* Thought: The current language of the user is...
9,Which of Wilhelm's direct male descendants liv...,", Friedrich Max Müller,",I cannot answer the question with the provided...,* Thought: The current language of the user is...


In [ ]:
f1s = []
ems = []
for a, agent_answer in zip(df["answer"].values, df['agent_answer'].values):

    f1s.append(compute_f1(a, agent_answer))
    ems.append(compute_exact(a, agent_answer))

df_metrics = pd.DataFrame()
df_metrics['F1'] = f1s
df_metrics['EM'] = ems
df_metrics.describe()

,F1,EM
count,50.000000,50.000000
mean,0.129971,0.020000
std,0.196608,0.141421
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.067308,0.000000
75%,0.164474,0.000000
max,1.000000,1.000000
